In [1]:
# Import Libraries
import math #Contains library for natural logirthm function
from scipy import optimize as optimize #Contains library for numerical solver

# Define solver function
def predicted(T_g, T_p1, flow_p, flow_h, R_SHX, R_GHX, L_SHX, L_GHX, rho_p, rho_h, cap_p, cap_h):
    '''
    This function calculates a numerical solution for pond outflow downstream of the geothermal cooling system (T_p2)
    based on other system parameters that must be defined:    
    T_g - Ground Temperature in units C
    T_p1 - Pond outflow temperature upstream of the geothermal system in units C
    flow_p - Pond flowrate in units L/s
    flow_h - Hydronic flowrate in units L/min
    R_SHX - Thermal resistance of the SHX in units mC/W
    R_GHX - Thermal resistance of the GHX in units mC/W
    L_GHX - Borehole length (depth of borehole(s) multiplied by number of boreholes) (m)
    L_SHX - Total length of pipe in the SHX that is immersed in the pond outflow (m)
    rho_p - Density of water kg/m3
    rho_h - Density of the heat transfer fluid kg/m3
    cap_p - Specific heat capacity of water J/(kgC)
    cap_h - Specific heat capacity of water J/(kgC)    
    '''
    # Change flow units to m3/s
    flow_p = flow_p*(1/1000)
    flow_h = flow_h*(1/60)*(1/1000)

     # There is a point discontinuity of these terms are the same - so if they are the same make them slightly different
    if (rho_p * flow_p * cap_p) == (rho_h * flow_h * cap_h):
        flow_p = flow_p * 1.001  

    # Function that defines the heat transfer as a function of the pond outlet temperature and other set parameters
    def q(T_p2):
        q_ = rho_p * flow_p * cap_p * (T_p1 - T_p2)
        return q_

    # Function that defines the hydronic system return temperature to the borehole as a function of the pond outlet temperature and other set parameters
    def T_h2(T_p2):
        T_h2_ = q(T_p2) * R_SHX / L_SHX + q(T_p2)/(2 * rho_h * flow_h * cap_h) + T_g
        return T_h2_

    # Function that defines the hydronic system supply temperature to the borehole as a function of the pond outlet temperature and other set parameters
    def T_h1(T_p2):
        T_h1_ = T_h2(T_p2) - q(T_p2) / (rho_h * flow_h * cap_h)
        if T_h1_ > T_g: #supply can't be cooler than ground itself...
            T_h1_x = T_h1_
        else:
            T_h1_x = T_g
        return T_h1_x

    # Function for average pond temperature   
    def Tp_ave(T_p2):
        return (T_p2+T_p1)/2

    # Function to be solved numerically for the pond outflow temperature (T_p2) 
    def y(T_p2):
        ln_numer = Tp_ave(T_p2) - T_h1(T_p2)
        ln_denom = Tp_ave(T_p2) - T_h2(T_p2)
        rat_denom = Tp_ave(T_p2) - T_h1(T_p2) - (Tp_ave(T_p2) - T_h2(T_p2))
        try:
            y_ =  q(T_p2) * R_SHX * math.log(ln_numer/ln_denom)/rat_denom - L_SHX
        except:
            return(-1)
        return y_

    # This is the denominator of the natural logarithm expression in y; it will be used in this function to identify when the ln argument blows up to infinity
    def ln_den(T_p2):
        y = Tp_ave(T_p2) - T_h2(T_p2)
        return y

    # This is when the argument of the natural logarithm blows up to infinity - need to avoid in our numerical solution
    def blows_up():
        blown_up = optimize.fsolve(ln_den, 15)
        return blown_up    

    # Need to provide an initial point for the numerical solver but there are vertical asymptotes that need to be avoided - this function is used to avoid them
    def post_blw_up_factor():
        if (flow_h < 3/15850):
            factor = 0.01
        else:
            factor = 0.0005
        return factor  

    # Define the initial geuss for the numerical solver
    post_blowup = blows_up()+post_blw_up_factor()

    # Solve the equations   
    Tp2_C = optimize.fsolve(y, post_blowup) # Pond outflow temp  
    Th1_C = T_h1(Tp2_C) # Hydronic system supply temp to surface water heat exchanger
    Th2_C = T_h2(Tp2_C) # Hydronic system return temp to borehole
    Tpave_C = (Tp2_C + T_p1)/2
    q_ = q(Tp2_C) # Cooling power in units ton; 1 ton = 12,000 kBtu/hr and 3,412 kBtu/hr = 1 kW
    
    # Confirm result is actually a solution - optimize.fsolve will still return a value even if it didn't find
    # the right solution. Therefore, the solution needs to be confirmed.
    T1 = Tpave_C - Th2_C 
    T2 = Tpave_C - Th1_C
    confirmed_correct = False
    try:
        y_result = q_ * R_SHX * math.log(T1/T2)/(T1-T2) - L_SHX
    except:
        y_result = -999    
    if (y_result < 0.001)&(y_result > -0.001):
        confirmed_correct = True
    
    result = {'Tp2_C':Tp2_C,
              'Th1_C':Th1_C,
              'Th2_C':Th2_C,
              'q_W':q_,
              'y_result':y_result,
              'confirmed_correct':confirmed_correct}
    
    return result  

In [2]:
result = predicted(T_g=15, T_p1=30, flow_p=0.4, flow_h=17, R_SHX=0.17, R_GHX=0.21, L_SHX=230, L_GHX=183, rho_p=1000, 
          rho_h=1000, cap_p=4186, cap_h=4186)
if result['confirmed_correct'] == True:
    print('The model successfully produced a solution.')
else:
    print('The model did not find a solution.')

The model successfully produced a solution.


In [3]:
print('For the provided parameters, the system provides ' + str(int(result['q_W'])) + ' W of cooling and the outflow \
temperature downstream of the goethermal system is ' + str(round(result['Tp2_C'][0],1)))

For the provided parameters, the system provides 8084 W of cooling and the outflow temperature downstream of the goethermal system is 25.2
